In [1]:
import pandas as pd
from typing import Optional, List
from core import system_msg, user_msg, assistant_msg, generic_call_stream
from qdrant_search import QdrantSearchEngine

In [2]:
from psql_models import get_session

In [3]:
engine = QdrantSearchEngine()

In [4]:
#await engine.initialize()
await engine.sync_from_postgres(get_session)

Created text index for field: topic_name
Created text index for field: summary
Created text index for field: details


In [7]:
pd.options.display.max_colwidth = 1000

In [9]:
query = 'incorporated'
main_results = await engine.search(
    query_text=query,
    limit=200,
    min_score=0.4,
)
pd.DataFrame(main_results)


,score,topic_name,summary,details,meeting_id,speaker_name,topic_type,timestamp,vector_scores,exact_matches
0,0.584086,Founders,Founders are individuals who establish a new business or organization.,Dmitry and Andy discuss the importance of targeting founders in their marketing efforts.,3540f190-1f83-4100-8d6f-e648752f336d,Dmitry Grankin,role,2024-09-26T12:02:28.836000,"{'topic_name': 0.58408636, 'summary': 0.29304211999999996}",[]
1,0.514227,municipality,Municipality refers to the local government authority that has an impact on the operations of Olga's company.,"Olga notes that the municipality has indicated it will not block certain tools, which could affect their operations.",3a964fa0-9463-4e7b-ad12-279ad64717d0,Olga Nemirovskaya,concept,2024-09-16T12:33:08.856000,{'topic_name': 0.51422703},[]
2,0.505460,venture capital,"Venture capital is a form of private equity financing, relevant to the discussion of funding and startup growth.",Dmitry references venture capital in the context of his previous experiences and the funding landscape for startups.,4ff12d56-f520-4391-a283-03d6365b49e6,Dmitry Grankin,investment,2024-09-09T10:46:17.724000,{'topic_name': 0.50545967},[]
3,0.501277,marketing agency,A marketing agency provides services to help businesses promote their products or services effectively.,"Olga runs a marketing agency that offers various services, including SMM and lead generation, to clients in different industries.",577a2c8f-f406-479d-9618-2fa48297e70d,Olga Miller,company,2024-09-06T10:30:31.135999,{'topic_name': 0.50127685},[]
4,0.454354,investor,Investors are mentioned in the context of the SAFE agreement and startup funding.,Dmitriy explains the role of investors in startups and how the SAFE agreement allows for investment without giving away equity immediately.,ed953dc9-002d-4202-905e-269cc982a3f3,Dmitriy Grankin,role,2024-09-30T15:36:01.794000,{'topic_name': 0.45435414},[]
5,0.451555,Founders for Founders,Founders for Founders is a topic mentioned in relation to a contract for freelancers.,"There is a discussion about sending a basic contract template to Lara for use with freelancers, along with information about Founders for Founders.",c57bb398-711b-42be-aa4f-9c75b0b4759b,Dmitriy Grankin,topic,2024-08-27T09:47:51.190000,{'topic_name': 0.45155472},[]
6,0.451555,Founders for Founders,Founders for Founders is another platform where outreach is planned.,Olga suggests that Dmitriy should reach out to Founders for Founders for potential collaboration.,cd43879a-0b16-4473-9744-04ae567626ff,Olga Nemirovskaya,organization,2024-08-26T19:14:18.976000,{'topic_name': 0.45155472},[]
7,0.451555,founders for founders,Founders for founders is a project that Dmitriy is involved in.,Dmitriy is tasked with reviewing and possibly rewriting the text related to this project as discussed in the meeting.,376586d0-be77-4b10-bbb5-075ea0fbc927,Dmitriy Grankin,project,2024-08-28T09:46:03.360000,"{'topic_name': 0.45155472, 'summary': 0.293486669}",[]
8,0.451555,Founders for Founders,Founders for Founders is a community platform for entrepreneurs and startup founders to connect and share resources.,Dmitriy suggests posting updates in the Founders for Founders community to engage with other entrepreneurs.,99ade764-fd60-4e9a-9621-66cc037cb865,Dmitriy Grankin,community,2024-08-27T19:34:48.996000,{'topic_name': 0.45155472},[]
9,0.432821,real estate,"Real estate is a business area discussed in the meeting, particularly in relation to the contact center project.",The contact center will focus on providing services related to ready rental businesses in the real estate market.,dd65ab42-1870-4158-9e2a-276fa5bda674,Igor Bessonov,topic,2024-10-10T16:34:40.500000,{'topic_name': 0.43282104},[]


In [7]:
query, instructions = query.split(':') if ':' in query else (query, '')
# Search for context using both methods with verbose mode for debugging
main_results = await engine.search(
    query_text=query,
    limit=200,
    min_score=0.2,
  #  exact_match_boost=2,  # Add boost parameter
   # verbose=False  # Can be set to True for debugging
)


In [8]:
pd.options.display.max_colwidth = 1000

In [9]:
pd.DataFrame(main_results)


""


In [ ]:

speaker_results = await engine.search_by_speaker(
    speaker_query=query,
    limit=200,
    min_score=0.4
)

In [31]:
speaker_results

[]

In [54]:

class ChatSession:
    def __init__(self, search_engine: QdrantSearchEngine, prompts):
        self.search_engine = search_engine
        self.prompts = prompts
        self.messages: List = []
        self.results = None
    
    def add_message(self, role: str, content: str):
        """Add a message to the conversation history"""
        if role == "system":
            self.messages.append(system_msg(content))
        elif role == "user":
            self.messages.append(user_msg(content))
        elif role == "assistant":
            self.messages.append(assistant_msg(content))
    
    async def process_user_message(self, query: str):
        """Process a user message and generate a response"""
        self.add_message("user", query)
        
        # Split query and instructions if present
        query, instructions = query.split(':') if ':' in query else (query, '')
        # Search for context using both methods with verbose mode for debugging
        main_results = await self.search_engine.search(
            query_text=query,
            limit=200,
            min_score=0.4,
        )
        
        speaker_results = await self.search_engine.search_by_speaker(
            speaker_query=query,
            limit=200,
            min_score=0.49
        )
        
        # Convert results to DataFrames with new score fields
        main_df = pd.DataFrame(main_results) if main_results else pd.DataFrame()
        speaker_df = pd.DataFrame(speaker_results) if speaker_results else pd.DataFrame()
        
        # Select relevant columns and combine results
        columns = ['topic_name', 'speaker_name', 'summary', 'details']
        score_columns = ['score', 'vector_scores', 'exact_matches']  # Add new score columns
        
        if len(main_df) > 0:
            main_df = main_df[columns + score_columns]  # Include score information
            main_df['source'] = 'main'
        else:
            main_df = pd.DataFrame(columns=columns + score_columns + ['source'])
        
        if len(speaker_df) > 0:
            speaker_df = speaker_df[columns + ['score']]  # Speaker search has simpler scoring
            speaker_df['source'] = 'speaker'
        else:
            speaker_df = pd.DataFrame(columns=columns + ['score', 'source'])
        
        # Combine and deduplicate results
        self.results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)
        
        # Sort by score
        if not self.results.empty:
            self.results = self.results.sort_values('score', ascending=False)
        # Search for context using both methods
        main_results = await self.search_engine.search(
            query_text=query,
            limit=200,
            min_score=0.4
        )
        
        speaker_results = await self.search_engine.search_by_speaker(
            speaker_query=query,
            limit=200,
            min_score=0.49
        )
        
        # Convert results to DataFrames
        main_df = pd.DataFrame(main_results) if main_results else pd.DataFrame()
        
        speaker_df = pd.DataFrame(speaker_results) if speaker_results else pd.DataFrame()
  
        columns = ['topic_name', 'speaker_name', 'summary', 'details','meeting_id']
        main_df = main_df[columns] if len(main_df) > 0 else pd.DataFrame(columns=columns)
        main_df['source'] = 'main'
        speaker_df = speaker_df[columns] if len(speaker_df) > 0 else pd.DataFrame(columns=columns)
        speaker_df['source'] = 'speaker'
        
        # Combine and deduplicate results
        self.results = pd.concat([main_df, speaker_df]).drop_duplicates().reset_index(drop=True)
        
        # Convert results to markdown format for context
        context = self.results[columns].to_markdown(index=False) if not self.results.empty else "No relevant context found."
        
        # Add context message
        context_msg = system_msg(f"Context: {context}")
        
        # Generate response
        messages_with_context = [
            system_msg(self.prompts.perplexity),
            system_msg('special instructions: ' + instructions),
            *self.messages,
            context_msg
        ]
        
        response = await generic_call_stream(messages_with_context)
        
        # Add assistant response
        self.add_message("assistant", response)
        
        return response

    def get_results(self) -> Optional[pd.DataFrame]:
        """Get the latest search results"""
        return self.results

In [55]:
# Initialize
from prompts import Prompts
prompts = Prompts()
search_engine = QdrantSearchEngine()
chat = ChatSession(search_engine, prompts)

In [56]:

# Process a message
response = await chat.process_user_message("Fireflies")

/tmp/ipykernel_1160475/4077025217.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)


TypeError: unhashable type: 'dict'

In [51]:
from IPython.display import Markdown
Markdown(response)

Fireflies is a meeting transcription tool that competes in the transcription software market. It has been noted for its AI capabilities, although some users, like Robert Hangu, have found its AI summaries to be inadequate compared to other tools like Vexa, which he prefers for its user-friendly interface and functionality[1][4][5]. 

Dmitry Grankin and other speakers discussed Fireflies in the context of its market presence and features, highlighting that it records and transcribes meetings, but operates differently from Vexa, which focuses on real-time transcription without using bots[2][3][6][7]. Overall, Fireflies is recognized as a significant competitor in the meeting transcription space, but user feedback suggests there are areas for improvement, particularly in AI summarization capabilities[4][5].

In [53]:

# Get results
results_df = chat.get_results()


,topic_name,speaker_name,summary,details,meeting_id,source
0,Fireflies,Dmitry Grankin,Fireflies is another competitor in the transcription software market.,Dmitry mentions Fireflies as part of the competitive landscape for his product.,3540f190-1f83-4100-8d6f-e648752f336d,main
1,Fireflies,Robert Hangu,Fireflies is a meeting transcription tool that Robert Hangu previously used before switching to Vexa.,Robert found Fireflies' AI summaries to be inadequate and prefers Vexa's more user-friendly interface and functionality.,4ff12d56-f520-4391-a283-03d6365b49e6,main
2,Fireflies,Dmitriy Grankin,"Fireflies is another competitor discussed in the meeting, noted for its market presence.",Dmitriy Grankin mentions Fireflies in the context of analyzing competitors' marketing strategies and user acquisition methods.,ae547dc7-22b0-4628-a715-4d0377adfc6c,main
3,Fireflies,Dmitry Grankin,Fireflies is mentioned as a competitor that provides transcription services for meetings.,The speakers discuss how Fireflies operates and the features it offers compared to their own products.,081d04c6-827d-4375-af68-9f8fbd50b707,main
4,Fireflies,Umar Lateef,Fireflies is another competitor in the meeting transcription market mentioned by the speakers.,"Umar notes that Fireflies is a market leader but operates differently from Vexa, which focuses on real-time transcription without using bots.",70cd7290-801a-4caf-9786-359cc6e16c60,main
5,Fireflies,Dmitriy Grankin,Fireflies is a meeting assistant tool that records and transcribes meetings.,"Dmitriy mentions Fireflies as a competitor and discusses user preferences for tools without bots, highlighting its features and user feedback.",d7d8d9e4-f349-42b3-9599-f9707f1405f8,main
6,Supernormal,Olga Nemirovskaya,Supernormal is a meeting assistant tool that competes with Fireflies and Notte.,"Olga includes Supernormal in her analysis of competitors using Google Ads, indicating a trend in the market.",d7d8d9e4-f349-42b3-9599-f9707f1405f8,main
7,bugs,Dmitriy Grankin,Issues identified in the software extension that need to be resolved in the upcoming build.,Dmitriy indicated that he had reported a lot of bugs that needed fixing in the new build.,74f0615a-57c2-4b0a-93e7-a3886f5766fd,main
8,Feedback on Vexa's usability and interface,Robert Hangu,"Robert provided positive feedback on Vexa's clean interface, contrasting it with Fireflies, which he found bloated and less user-friendly. He appreciates Vexa's flexibility and ease of use.",Robert has used Vexa for multiple calls and finds its interface straightforward and efficient. He prefers it over Fireflies due to its clean design and the ability to interact with the transcript easily. He has not encountered any significant issues with the interface so far.,4ff12d56-f520-4391-a283-03d6365b49e6,main
9,Discussion on the effectiveness of AI tools for summarizing meetings,Robert Hangu,"Robert shared his experience with Vexa and Fireflies, highlighting the importance of having a tool that allows for interaction with the transcript rather than just providing a static summary.","Robert mentioned that he previously used Fireflies but found its AI summaries lacking. He now uses Vexa to extract specific information from call transcripts, which he finds much more effective. He emphasized the need for tools that allow for open-ended questions to gain deeper insights.",4ff12d56-f520-4391-a283-03d6365b49e6,main


In [ ]:
results_df